In [8]:
# from notebookinit import *
import os, json

In [9]:
config_path = os.path.normpath(os.path. os.path.join(os.getcwd(), '..', '..', '..',  'config.json'))
config_path

'/home/mib07150/git/private/gyals/config.json'

In [10]:
!pwd

/home/mib07150/git/private/gyals/notebook/dev/archive


In [12]:
config = json.loads(open(config_path).read())
stream_count = len(config['streams'])
print(f'got {stream_count} streams')
for stream in range(stream_count):
    print(f'stream {stream}:')
    for key in config["streams"][stream].keys():
        if key == 'password':
            print(f'{key:>30} -> {"*" * len(config["streams"][stream][key])}')
        else:
            print(f'{key:>30} -> {config["streams"][stream][key]}')

data_path = os.path.normpath(os.path.join(os.getcwd(), '..', '..', config['data-folder']))
# print(f'data path: {data_path}')
os.makedirs(data_path, exist_ok=True)

got 1 streams
stream 0:
           mqtt-broker-address -> 192.168.1.2
              mqtt-broker-port -> 1883
                          user -> aaa
                      password -> ***
                         topic -> #
                        prefix -> aaa
                        suffix -> 
    file-rotation-time-seconds -> 600


In [6]:
# for each stream, create a folder with that prefix. If there is no prefix, use stream number
for stream in range(stream_count):
    folder_prefix = config['streams'][stream]['prefix']
    if folder_prefix == '':
        folder_prefix = str(stream)
    folder_suffix = config['streams'][stream]['suffix']
    folder_name = folder_prefix + folder_suffix
    stream_path = os.path.join(data_path, folder_name)
    print(f'creating {stream_path}')
    os.makedirs(stream_path, exist_ok=True)

creating /home/mib07150/git/STL/george-yals/datalog/all


In [7]:
from paho.mqtt import client as mqtt
import sqlite3
import pandas

global SharedState
SharedState = {}
SharedState['totalMessageCount'] = 0
SharedState['streams'] = []
messageCount=0
def make_onmessage_callback(stream_idx, intendedTopic, stream_path="", logRotationTime=600):

    os.makedirs(stream_path, exist_ok=True)
    global SharedState    

    SharedState['streams'][stream_idx] = dict(
        messageCount=0, 
        intendedTopic=intendedTopic, 
        lastMessage=None, 
        sqlite_cursor=None, 
        sqlite_connection=None, 
        next_rotation_time=0,
        )

    def curried_on_message(client, userdata, msg):
        global SharedState

        def renew_database_link():
            file_name = f'{int(time.time())}.sqlite'
            sqlite_db_path = os.path.join(stream_path, file_name) # future: make this incremental by start timestamp
            print(f'{stream_idx=} creating new file {sqlite_db_path}')                          
            sqlite_connection = sqlite3.connect(sqlite_db_path)
            sqlite_cursor = sqlite_connection.cursor()
            sqlite_cursor.execute('CREATE TABLE IF NOT EXISTS data (payload TEXT, reception_timestamp INTEGER)')
            sqlite_connection.commit()
            SharedState['streams'][stream_idx]['sqlite_cursor'] = sqlite_cursor
            SharedState['streams'][stream_idx]['sqlite_connection'] = sqlite_connection
            next_rotation_time = time.time() + logRotationTime
            SharedState['streams'][stream_idx]['next_rotation_time'] = next_rotation_time
            return sqlite_cursor, sqlite_connection


        if SharedState['streams'][stream_idx]['sqlite_cursor'] is None:
            # create the cursor and connection. this is because the object must be created and used in the same thread.
            sqlite_cursor, sqlite_connection = renew_database_link()                          
        else:
            # load the cursor and connection from the shared state. this is because the object must be created and used in the same thread.
            sqlite_cursor = SharedState['streams'][stream_idx]['sqlite_cursor']
            sqlite_connection = SharedState['streams'][stream_idx]['sqlite_connection']
            # check for rotation time
            next_rotation_time = SharedState['streams'][stream_idx]['next_rotation_time']
            if time.time() > next_rotation_time:
                # rotate the log
                sqlite_connection.commit()
                sqlite_cursor.close()
                sqlite_connection.close()
                SharedState['streams'][stream_idx]['sqlite_cursor'] = None
                SharedState['streams'][stream_idx]['sqlite_connection'] = None
                # re-open the log with next file name
                sqlite_cursor, sqlite_connection = renew_database_link()



        # note to the purist: I am aware of the GIL. I am not worried about it here: 
        # on my machine, GIL with modification of global state, switches approx. 20e6 times per second, so I am not worried about the performance impact of this code
        SharedState['totalMessageCount'] += 1
        SharedState['streams'][stream_idx]['messageCount'] +=1
        topic = msg.topic
        payload = msg.payload.decode('utf-8')
        SharedState['streams'][stream_idx]['lastMessage'] = payload
        timestamp_unix = int(1e6*pandas.Timestamp.now().timestamp()) # microseconds since epoch
        SharedState['streams'][stream_idx]['lastRxTimestamp_unix'] = timestamp_unix
        # # use the cursor to write to the database
        if sqlite_cursor is not None:
            sqlite_cursor.execute('INSERT INTO data VALUES (?, ?)', (payload, timestamp_unix))
            # sqlite_connection.commit()  # no need to commit as this is done in the rotation code

    return curried_on_message

def on_connect(client, userdata, flags, rc):
    print(f'connected to mqtt broker with result code {rc} and flags {flags}')

# for each stream, create mqtt client and subscribe to topic
stream_clients = []
SharedState['streams'] = []

for stream_idx in range(stream_count):
    stream_config = config['streams'][stream_idx]


    # prepare path to save the data to:
    folder_prefix = config['streams'][stream]['prefix']
    if folder_prefix == '':
        folder_prefix = str(stream)
    folder_suffix = config['streams'][stream]['suffix']
    folder_name = folder_prefix + folder_suffix
    stream_path = os.path.join(data_path, folder_name)

    # create the mqtt client and pass on the curried on_message function    
    
    mqtt_client = mqtt.Client()
    mqtt_client.on_connect = on_connect    
    SharedState['streams'].append(dict())
    
    # i think this pattern is called "making a closure", that is, a function that returns a function that closes over some variables provided from the outer function
    this_on_message = make_onmessage_callback(stream_idx, intendedTopic=stream_config['topic'], stream_path=stream_path, logRotationTime=stream_config['file-rotation-time-seconds'])
    mqtt_client.on_message = this_on_message

    mqtt_client.username_pw_set(stream_config['user'], stream_config['password'])
    mqtt_client.connect(stream_config['mqtt-broker-address'], stream_config['mqtt-broker-port'], 60)
    print(f'subscribing client {stream_idx} to {stream_config["topic"]}')
    mqtt_client.subscribe(stream_config["topic"])

    mqtt_client.loop_start()
    stream_clients.append(mqtt_client)
    


subscribing client 0 to #
connected to mqtt broker with result code 0 and flags {'session present': 0}


In [8]:
time.sleep(30)
stream_clients[0].loop_stop()


stream_idx=0 creating new file /home/mib07150/git/STL/george-yals/datalog/all/1679590570.sqlite


time.time()=1679590580.6382027 and next_rotation_time=1679590580.6376586 -> rotating log for stream 0
stream_idx=0 creating new file /home/mib07150/git/STL/george-yals/datalog/all/1679590580.sqlite
time.time()=1679590590.6628237 and next_rotation_time=1679590590.6625922 -> rotating log for stream 0
stream_idx=0 creating new file /home/mib07150/git/STL/george-yals/datalog/all/1679590590.sqlite
time.time()=1679590600.7069607 and next_rotation_time=1679590600.687008 -> rotating log for stream 0
stream_idx=0 creating new file /home/mib07150/git/STL/george-yals/datalog/all/1679590600.sqlite


In [9]:
print(SharedState['streams'])
print(SharedState['totalMessageCount'])

[{'messageCount': 9329, 'intendedTopic': '#', 'lastMessage': '{\n\t"_type" : "squirrel::Data",\n\t"data" : \n\t{\n\t\t"backscatterSignal" : 2965.0,\n\t\t"integration_time" : 0.0099942395463585854,\n\t\t"photonCount" : 22799.0,\n\t\t"range" : 5.707371711730957,\n\t\t"range_width" : 1.9986163377761841,\n\t\t"servo_position" : \n\t\t{\n\t\t\t"beam" : \n\t\t\t[\n\t\t\t\t0.13010817640277336,\n\t\t\t\t-0.089549283009337885\n\t\t\t],\n\t\t\t"p" : \n\t\t\t[\n\t\t\t\t9.171137809753418,\n\t\t\t\t1.4592573642730713\n\t\t\t],\n\t\t\t"units" : "rad",\n\t\t\t"v" : \n\t\t\t[\n\t\t\t\t2.3356640338897705,\n\t\t\t\t0.54721271991729736\n\t\t\t]\n\t\t},\n\t\t"signal" : 11693.0,\n\t\t"source" : "returnPulse",\n\t\t"spectrum_down" : \n\t\t[\n\t\t\t327.91152954101562,\n\t\t\t363.82644653320312,\n\t\t\t462.25042724609375,\n\t\t\t382.44924926757812,\n\t\t\t292.3037109375,\n\t\t\t362.73358154296875,\n\t\t\t376.96334838867188,\n\t\t\t381.35147094726562,\n\t\t\t328.99530029296875,\n\t\t\t363.82644653320312,\n\t\t

In [10]:
# stream_clients[0].loop_stop()


In [11]:
SharedState['streams'][stream_idx]['next_rotation_time']

1679590610.7270596

In [12]:
time.time()

1679590601.2566917

In [13]:
# # option 1: use sqlite to store the data

# sqlite_db_path = os.path.join(data_path, 'data.sqlite')
# conn = sqlite3.connect(sqlite_db_path)
# c = conn.cursor()
# c.execute('CREATE TABLE IF NOT EXISTS data (payload TEXT, reception_timestamp INTEGER)')
# conn.commit()
# # to insert data, use
# # c.execute('INSERT INTO data VALUES (?, ?)', (payload, reception_timestamp))
# # conn.commit()

# # option 2: use pandas to store the data
# import pandas as pd
# data = pd.DataFrame(columns=['payload', 'reception_timestamp'])





In [14]:
print(SharedState['streams'][0]['lastRxTimestamp_unix'])

1679590600727082


In [15]:
# benchmark the GIL with write to a global state.
if 1==0:
    import time
    import threading

    thread_count = 8
    loop_count = 20000000
    instructions_per_loop = 3

    # define a function that does some work
    global j
    j = 0 
    def do_work():
        global j
        for i in range(loop_count):
            j+=i
            j*=i
            j%=(i+1)

        print(f'j={j}')

    # define a function that creates and starts 3 threads
    def benchmark():
        threads = []
        for i in range(thread_count):
            t = threading.Thread(target=do_work)
            threads.append(t)
            t.start()

        # wait for all threads to finish
        for t in threads:
            t.join()

    # measure the time it takes to run the benchmark function
    start_time = time.monotonic()
    benchmark()
    end_time = time.monotonic()

    # calculate the maximum command throughput across all threads
    total_commands = thread_count * loop_count * instructions_per_loop
    elapsed_time = end_time - start_time
    throughput = total_commands / elapsed_time

    print(f"Elapsed time: {elapsed_time:.3f} seconds")
    print(f"Throughput: {throughput:.3f} commands per second")